# importing required packages

- import numpy for linear algebra
- import pandas for loading the dataset, data processing
- import matplotlib and seaborn for visualization

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# import matplotlib
# import scipy as sp
# from subprocess import check_output # #print(check_output(["ls", "../input"]).decode("utf8"))

In [2]:
import boto3
import s3fs

In [3]:
import os

# Load the datasets

- to read the file from AWS

In [4]:
client = boto3.client('s3')

In [5]:
path = 's3://nyc-taxi-system/Final_dataset.csv'

In [6]:
bulk_df = pd.read_csv(path)
bulk_df.head(10)

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,total_amount,congestion_surcharge,PU_zone,PU_borough,PU_latitude,PU_longitude,DO_zone,DO_borough,DO_latitude,DO_longitude
0,0,1.0,2020-01-07 21:38:40,2020-01-07 21:45:13,2.0,1.00,1.0,N,249,113,...,12.85,2.5,West Village,Manhattan,40.734186,-74.005580,Greenwich Village North,Manhattan,40.734299,-74.000573
1,1,2.0,2020-01-24 20:09:35,2020-01-24 20:52:36,1.0,3.99,1.0,N,144,142,...,35.76,2.5,Little Italy/NoLiTa,Manhattan,40.721006,-73.994702,Lincoln Square East,Manhattan,40.775707,-73.991420
2,2,2.0,2020-01-15 13:37:54,2020-01-15 13:40:47,1.0,0.43,1.0,N,238,238,...,9.49,2.5,Upper West Side North,Manhattan,40.781960,-73.981243,Upper West Side North,Manhattan,40.781960,-73.981243
3,3,2.0,2020-01-16 16:22:45,2020-01-16 16:28:19,1.0,0.60,1.0,N,164,137,...,10.80,2.5,Midtown South,Manhattan,40.749842,-73.984251,Kips Bay,Manhattan,40.739546,-73.977083
4,4,1.0,2020-01-01 14:11:40,2020-01-01 14:12:28,4.0,0.00,1.0,N,132,132,...,3.30,0.0,JFK Airport,Queens,40.642948,-73.779373,JFK Airport,Queens,40.642948,-73.779373
5,5,2.0,2020-01-22 23:33:47,2020-01-22 23:39:45,2.0,1.45,1.0,N,142,237,...,12.42,2.5,Lincoln Square East,Manhattan,40.775707,-73.991420,Upper East Side South,Manhattan,40.779468,-73.958265
6,6,1.0,2020-01-04 16:12:29,2020-01-04 16:31:23,2.0,3.90,1.0,N,90,239,...,23.30,2.5,Flatiron,Manhattan,40.741059,-73.989642,Upper West Side South,Manhattan,40.769244,-73.990767
7,7,1.0,2020-01-31 18:49:37,2020-01-31 19:03:00,1.0,2.90,1.0,N,230,236,...,15.80,2.5,Times Sq/Theatre District,Manhattan,40.758168,-73.985525,Upper East Side North,Manhattan,40.764873,-73.961511
8,8,2.0,2020-01-12 07:09:42,2020-01-12 07:14:05,2.0,1.19,1.0,N,263,140,...,11.16,2.5,Yorkville West,Manhattan,43.670470,-79.387252,Lenox Hill East,Manhattan,40.765177,-73.961595
9,9,2.0,2020-01-01 10:49:09,2020-01-01 10:52:21,1.0,0.77,1.0,N,42,42,...,5.30,0.0,Central Harlem North,Manhattan,40.799218,-73.954900,Central Harlem North,Manhattan,40.799218,-73.954900


- remove the first column which is index column saved in dataset from prior sampling process

In [7]:
bulk_df.shape

(1200911, 27)

In [8]:
bulk_df.drop(['Unnamed: 0'],axis=1,inplace=True)

### to take 10% of total dataset as a sample

In [9]:
df = bulk_df.sample(frac=0.1)

In [10]:
df.shape

(120091, 26)

# Exploratory Data Analysis ( EDA )

In [11]:
from pyspark.sql import SparkSession

In [12]:
spark=SparkSession.builder.appName('Dataframe').getOrCreate()

In [13]:
spark

In [14]:
from pyspark.sql.types import *

In [15]:
schema=StructType([
    StructField("VendorID", FloatType(), True ),
    StructField("tpep_pickup_datetime", StringType(), True ),
    StructField("tpep_dropoff_datetime", StringType(), True ),
    StructField("passenger_count", FloatType(), True ),
    StructField("trip_distance", FloatType(), True ),
    StructField("RatecodeID", FloatType(), True ),
    StructField("store_and_fwd_flag", StringType(), True ),
    StructField("PULocationID", IntegerType(), True ),
    StructField("DOLocationID", IntegerType(), True ),
    StructField("payment_type", FloatType(), True ),
    StructField("fare_amount", FloatType(), True ),
    StructField("extra", FloatType(), True ),
    StructField("mta_tax", FloatType(), True ),
    StructField("tip_amount", FloatType(), True ),
    StructField("tolls_amount", FloatType(), True ),
    StructField("improvement_surcharge", FloatType(), True ),
    StructField("total_amount", FloatType(), True ),
    StructField("congestion_surcharge", FloatType(), True ),
    StructField("PU_zone ", StringType(), True ),
    StructField("PU_borough", StringType(), True ),
    StructField("PU_latitude", FloatType(), True ),
    StructField("PU_longitude", FloatType(), True ),
    StructField("DO_zone", StringType(), True ),
    StructField("DO_borough", StringType(), True ),
    StructField("DO_latitude", FloatType(), True ),
    StructField("DO_longitude", FloatType(), True )
])

In [16]:
# spark.read.format('csv').options(header='true', inferSchema='True').load('s3://nyc-taxi-system/Final_dataset.csv')

In [17]:
df_pyspark = spark.createDataFrame(df, schema)

In [18]:
df_pyspark.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'PU_zone ',
 'PU_borough',
 'PU_latitude',
 'PU_longitude',
 'DO_zone',
 'DO_borough',
 'DO_latitude',
 'DO_longitude']

In [19]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [20]:
df_pyspark.printSchema()

root
 |-- VendorID: float (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: float (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- RatecodeID: float (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: float (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- congestion_surcharge: float (nullable = true)
 |-- PU_zone : string (nullable = true)
 |-- PU_borough: string (nullable = true)
 |-- PU_latitude: float (nullable = true)
 |-- PU_longitude: float (nullable = true)
 |-- DO_zone: string (nu

In [21]:
df_pyspark.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+--------------------+----------+-----------+------------+--------------------+----------+-----------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|            PU_zone |PU_borough|PU_latitude|PU_longitude|             DO_zone|DO_borough|DO_latitude|DO_longitude|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+-----------------

In [22]:
df_pyspark.show(vertical=True)

-RECORD 0-------------------------------------
 VendorID              | 2.0                  
 tpep_pickup_datetime  | 2020-01-13 12:52:07  
 tpep_dropoff_datetime | 2020-01-13 12:59:40  
 passenger_count       | 5.0                  
 trip_distance         | 1.62                 
 RatecodeID            | 1.0                  
 store_and_fwd_flag    | N                    
 PULocationID          | 142                  
 DOLocationID          | 237                  
 payment_type          | 1.0                  
 fare_amount           | 7.5                  
 extra                 | 0.0                  
 mta_tax               | 0.5                  
 tip_amount            | 2.16                 
 tolls_amount          | 0.0                  
 improvement_surcharge | 0.3                  
 total_amount          | 12.96                
 congestion_surcharge  | 2.5                  
 PU_zone               | Lincoln Square East  
 PU_borough            | Manhattan            
 PU_latitude 

In [23]:
## from pyspark.sql.functions import *

In [24]:
## column_names = ["VendorID","tpep_pickup_datetime","tpep_dropoff_datetime","passenger_count","trip_distance","RatecodeID","store_and_fwd_flag","PULocationID","DOLocationID","payment_type","fare_amount","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","congestion_surcharge","PU_zone","PU_borough","PU_latitude","PU_longitude","DO_zone","DO_borough","DO_latitude","DO_longitude"]

In [25]:
## df_final = df_pyspark.toDF(*column_names)

In [26]:
## df_final.columns

In [27]:
## df_final.show()

In [28]:
## df_final.select('VendorID').distinct().show()

In [29]:
## df_pyspark=spark.read.csv('s3://nyc-taxi-system/Final_dataset.csv',header=True,inferSchema=True)

### shape

- returns number of rows and columns

In [30]:
## df.shape()

In [31]:
print((df_pyspark.count(), len(df_pyspark.columns)))

(120091, 26)


- 95074 rows and 26 columns

### columns in dataset

- print the columns names

In [32]:
## df.columns

In [33]:
df_pyspark.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'PU_zone ',
 'PU_borough',
 'PU_latitude',
 'PU_longitude',
 'DO_zone',
 'DO_borough',
 'DO_latitude',
 'DO_longitude']

- printing first 5 records

In [34]:
## df.head()

In [35]:
df_pyspark.head(n=5)

[Row(VendorID=2.0, tpep_pickup_datetime='2020-01-13 12:52:07', tpep_dropoff_datetime='2020-01-13 12:59:40', passenger_count=5.0, trip_distance=1.6200000047683716, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=142, DOLocationID=237, payment_type=1.0, fare_amount=7.5, extra=0.0, mta_tax=0.5, tip_amount=2.1600000858306885, tolls_amount=0.0, improvement_surcharge=0.30000001192092896, total_amount=12.960000038146973, congestion_surcharge=2.5, PU_zone ='Lincoln Square East', PU_borough='Manhattan', PU_latitude=40.77570724487305, PU_longitude=-73.99141693115234, DO_zone='Upper East Side South', DO_borough='Manhattan', DO_latitude=40.77946853637695, DO_longitude=-73.95826721191406),
 Row(VendorID=2.0, tpep_pickup_datetime='2019-09-16 17:13:30', tpep_dropoff_datetime='2019-09-16 17:24:38', passenger_count=5.0, trip_distance=2.940000057220459, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=137, DOLocationID=232, payment_type=1.0, fare_amount=11.0, extra=1.0, mta_tax=0.5, tip_amount=

### Demographic information of Customer & Vendor

    1. 'VendorID' – code indicating the TPER ( Taxicab and Livery Passenger ENchancement Programs (TPEP ) provider that provided the record	
	2. 'tpep_pickup_datetime' – pick up date and time 
	3. 'tpep_dropoff_datetime' – drop off date and time
    4. 'passenger_count' – passenger count 
	5. 'trip_distance' – elapsed trip distance in miles reported by the taximeter 
	6. 'RatecodeID' – final rate code effect at the end of the trip
		1 – Standard Rate
		2 – JFK
		3 – Newark
		4 – Nassau
		5 – Negotiated fare
		6 – group ride
	7. 'store_and_fwd_flag' – flag indicated whether the trip record was held in vehicle
		Y – store and forward trip
		N – not a store and forward trip
    8. 'PULocationID' – TLC Taxi zone in which taximeter was engaged 
	9. 'DOLocationID' – TLC Taxi zone in whoch taximeter was disengaged 
	10. 'payment_type' – numeric code signifying the payment mode
		1 – credit card
		2 – cash
		3 – no charge
		4 – dispute
		5 – unknown
		6 – voided trip
 	11. 'fare_amount' – time and distance fare calculated by the meter 
	12. 'extra' – miscellaneous extras and surcharges.
		1. Rush charge – 0.5 USD
		2. Overnight charge – 1 USD
    13. 'mta_tax' – 0.50 MTA tax, automatically added 
	14. 'tip_amount' – tip amount. field is automatically populated for credit card tips., cash tips are not included. 
	15. 'tolls_amount' – total amount of all tolls paid in trip 
	16. 'improvement_surcharge' – 0.30 imporvement surcharge assessed trips at the flag drop.
        17. 'total_amount' – total amount charged to passengers. Does not include cash tips. 
	18. 'congestion_surcharge' – additional charge added to base charge during unusal events like strikes, bad winter, major port fires.
    19. zone_x - pick up zone
    20. borough_x - pick up borough
    21. PUlatitude - pick up latitude
    22. PUlongitude - pick up longitude
    23. zone_y - drop off zone
    24. borough_y - drop off borough
    25. DOlatitude - drop off latitude
    26. DOlongitude - drop off longitude

### info()

- it will tell you how many entries you have, the names of each column, the data type of each column, and how many non-null values you have in each column

In [36]:
# df.info()

In [37]:
from pyspark.sql.functions import *

In [38]:
df_pyspark.printSchema()

root
 |-- VendorID: float (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: float (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- RatecodeID: float (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: float (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- congestion_surcharge: float (nullable = true)
 |-- PU_zone : string (nullable = true)
 |-- PU_borough: string (nullable = true)
 |-- PU_latitude: float (nullable = true)
 |-- PU_longitude: float (nullable = true)
 |-- DO_zone: string (nu

### Descriptive Statistics

- Descriptive statistics is a helpful way to understand characteristics of your data and to get a quick summary of it. 
- The describe function applies basic statistical computations on the dataset like extreme values, count of data points standard deviation etc. 
- Any missing value or NaN value is automatically skipped. 

In [39]:
# df.describe()

In [40]:
df_pyspark.describe().toPandas()

,summary,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,total_amount,congestion_surcharge,PU_zone,PU_borough,PU_latitude,PU_longitude,DO_zone,DO_borough,DO_latitude,DO_longitude
0,count,120091,120091,120091,120091,120091,120091,120091,120091,120091,...,120091,120091,120091,120091,120091,120091,120091,120091,120091,120091
1,mean,NaN,None,None,NaN,4.435011107988762,NaN,NaN,163.05753137204286,159.5821502027629,...,18.528659277342427,2.1594728164475274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,stddev,NaN,None,None,NaN,345.1919073034631,NaN,NaN,67.4241103362905,71.83540590581744,...,14.173633668144985,0.881234064211357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,min,1.0,2009-01-01 00:40:09,2009-01-01 00:44:25,0.0,-21.36,1.0,N,1,1,...,-154.42,-2.5,Allerton,Bronx,40.508476,-79.43939,Allerton,Bronx,40.508476,-79.43939
4,max,NaN,2021-07-31 23:56:52,2021-08-01 15:45:16,NaN,84813.76,NaN,Y,265,265,...,477.8,2.5,Yorkville West,Staten Island,NaN,NaN,Yorkville West,Staten Island,NaN,NaN


In [41]:
df_pyspark.summary().toPandas()

,summary,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,total_amount,congestion_surcharge,PU_zone,PU_borough,PU_latitude,PU_longitude,DO_zone,DO_borough,DO_latitude,DO_longitude
0,count,120091,120091,120091,120091,120091,120091,120091,120091,120091,...,120091,120091,120091,120091,120091,120091,120091,120091,120091,120091
1,mean,NaN,None,None,NaN,4.435011107988762,NaN,NaN,163.05753137204286,159.5821502027629,...,18.528659277342427,2.1594728164475274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,stddev,NaN,None,None,NaN,345.1919073034631,NaN,NaN,67.4241103362905,71.83540590581744,...,14.173633668144985,0.881234064211357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,min,1.0,2009-01-01 00:40:09,2009-01-01 00:44:25,0.0,-21.36,1.0,N,1,1,...,-154.42,-2.5,Allerton,Bronx,40.508476,-79.43939,Allerton,Bronx,40.508476,-79.43939
4,25%,1.0,None,None,1.0,1.0,1.0,NaN,114,107,...,11.15,2.5,NaN,NaN,40.74106,-73.99142,NaN,NaN,40.739548,-73.99136
5,50%,2.0,None,None,1.0,1.7,1.0,NaN,162,161,...,14.3,2.5,NaN,NaN,40.759823,-73.98408,NaN,NaN,40.759823,-73.98056
6,75%,2.0,None,None,2.0,3.18,1.0,NaN,234,234,...,20.16,2.5,NaN,NaN,40.775707,-73.96151,NaN,NaN,40.775715,-73.95947
7,max,NaN,2021-07-31 23:56:52,2021-08-01 15:45:16,NaN,84813.76,NaN,Y,265,265,...,477.8,2.5,Yorkville West,Staten Island,NaN,NaN,Yorkville West,Staten Island,NaN,NaN


In [42]:
### Measures of Central Tendency

- mean of values of column

In [43]:
# df.mean()

In [44]:
df_pyspark.select(*[mean(c).alias(c) for c in df_pyspark.columns]).show(vertical=True)

-RECORD 0------------------------------------
 VendorID              | NaN                 
 tpep_pickup_datetime  | null                
 tpep_dropoff_datetime | null                
 passenger_count       | NaN                 
 trip_distance         | 4.435011107988762   
 RatecodeID            | NaN                 
 store_and_fwd_flag    | NaN                 
 PULocationID          | 163.05753137204286  
 DOLocationID          | 159.5821502027629   
 payment_type          | NaN                 
 fare_amount           | 12.890548756377127  
 extra                 | 1.0413217476791867  
 mta_tax               | 0.492442814199133   
 tip_amount            | 2.0266098189788204  
 tolls_amount          | 0.32889533494177786 
 improvement_surcharge | 0.2973245407135336  
 total_amount          | 18.528659277342427  
 congestion_surcharge  | 2.1594728164475274  
 PU_zone               | NaN                 
 PU_borough            | NaN                 
 PU_latitude           | NaN      

- median of respective columns

In [45]:
# df.median()

In [46]:
df_pyspark.agg(*[percentile_approx(c, 0.5).alias(c) for c in df_pyspark.columns]).show(vertical=True)

-RECORD 0--------------------------
 VendorID              | 2.0       
 tpep_pickup_datetime  | null      
 tpep_dropoff_datetime | null      
 passenger_count       | 1.0       
 trip_distance         | 1.7       
 RatecodeID            | 1.0       
 store_and_fwd_flag    | NaN       
 PULocationID          | 162       
 DOLocationID          | 161       
 payment_type          | 1.0       
 fare_amount           | 9.0       
 extra                 | 0.5       
 mta_tax               | 0.5       
 tip_amount            | 1.86      
 tolls_amount          | 0.0       
 improvement_surcharge | 0.3       
 total_amount          | 14.3      
 congestion_surcharge  | 2.5       
 PU_zone               | NaN       
 PU_borough            | NaN       
 PU_latitude           | 40.759823 
 PU_longitude          | -73.98408 
 DO_zone               | NaN       
 DO_borough            | NaN       
 DO_latitude           | 40.759823 
 DO_longitude          | -73.98056 



In [47]:
# df_pyspark.agg(expr("approx_percentile(array(0.5))")).show()

- mode of values of columns

In [48]:
# df.mode()

In [50]:
# [[c, df_pyspark.groupby(c).count().orderBy("count", ascending=False).first()[0]] for c in df_pyspark.columns]

### to check the null values

In [51]:
# df.isnull().sum()

In [52]:
# df_pyspark.select([count(col(c).isNull()).alias(c) for c in df_pyspark.columns]).show()

### checking the missing values

In [54]:
df_pyspark.select(*[count(isnan(c) | isnull(c)).alias(c) for c in df_pyspark.columns]).show(vertical=True)

-RECORD 0-----------------------
 VendorID              | 120091 
 tpep_pickup_datetime  | 120091 
 tpep_dropoff_datetime | 120091 
 passenger_count       | 120091 
 trip_distance         | 120091 
 RatecodeID            | 120091 
 store_and_fwd_flag    | 120091 
 PULocationID          | 120091 
 DOLocationID          | 120091 
 payment_type          | 120091 
 fare_amount           | 120091 
 extra                 | 120091 
 mta_tax               | 120091 
 tip_amount            | 120091 
 tolls_amount          | 120091 
 improvement_surcharge | 120091 
 total_amount          | 120091 
 congestion_surcharge  | 120091 
 PU_zone               | 120091 
 PU_borough            | 120091 
 PU_latitude           | 120091 
 PU_longitude          | 120091 
 DO_zone               | 120091 
 DO_borough            | 120091 
 DO_latitude           | 120091 
 DO_longitude          | 120091 



In [64]:
import pyspark.sql.functions as F

In [68]:
# result = df_pyspark.select('VendorID').join(
#     df_pyspark.select("nan"), 
#     F.array_contains("nan", F.col('VendorID'))
# )

In [71]:
# result = df_pyspark.select(array_contains("VendorID", "nan")).collect()

In [78]:
# df_pyspark.filter(df_pyspark.RatecodeID.contains('nan')).count()

In [79]:
# df_pyspark.select(array_contains(df_pyspark.VendorID, )).collect()

### unique elements

In [55]:
# df.nunique()

In [56]:
# df_pyspark.dropDuplicates((["VendorID","tpep_pickup_datetime","tpep_dropoff_datetime","passenger_count","trip_distance","RatecodeID","store_and_fwd_flag","PULocationID","DOLocationID","payment_type","fare_amount","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","congestion_surcharge","PU_zone","PU_borough","PU_latitude","PU_longitude","DO_zone","DO_borough","DO_latitude","DO_longitude"])).select("VendorID","tpep_pickup_datetime","tpep_dropoff_datetime","passenger_count","trip_distance","RatecodeID","store_and_fwd_flag","PULocationID","DOLocationID","payment_type","fare_amount","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","congestion_surcharge","PU_zone","PU_borough","PU_latitude","PU_longitude","DO_zone","DO_borough","DO_latitude","DO_longitude").show()

In [57]:
df_pyspark.agg(*(countDistinct(col(c)).alias(c) for c in df_pyspark.columns)).show(vertical=True)

-RECORD 0-----------------------
 VendorID              | 3      
 tpep_pickup_datetime  | 119874 
 tpep_dropoff_datetime | 119873 
 passenger_count       | 9      
 trip_distance         | 2529   
 RatecodeID            | 7      
 store_and_fwd_flag    | 3      
 PULocationID          | 247    
 DOLocationID          | 256    
 payment_type          | 5      
 fare_amount           | 2859   
 extra                 | 34     
 mta_tax               | 4      
 tip_amount            | 1386   
 tolls_amount          | 131    
 improvement_surcharge | 3      
 total_amount          | 4513   
 congestion_surcharge  | 4      
 PU_zone               | 244    
 PU_borough            | 7      
 PU_latitude           | 243    
 PU_longitude          | 245    
 DO_zone               | 253    
 DO_borough            | 7      
 DO_latitude           | 252    
 DO_longitude          | 254    



In [58]:
df_pyspark.select('VendorID').distinct().collect()

[Row(VendorID=2.0), Row(VendorID=1.0), Row(VendorID=nan)]

In [59]:
df_pyspark.select('RatecodeID').distinct().collect()

[Row(RatecodeID=5.0),
 Row(RatecodeID=2.0),
 Row(RatecodeID=3.0),
 Row(RatecodeID=1.0),
 Row(RatecodeID=4.0),
 Row(RatecodeID=nan),
 Row(RatecodeID=99.0)]

#### Conclusion:


- there are 3 vendors active in NYC
- The fare_amount variable has negative value as minimum value which is unusual
- The passenger_count variable having 0 as minimum no of passengers and maximum as 9
- The pickup and dropoff zone's location_Id shouldn't be greter than 263

# Data Cleansing

#### analysis of records having null values

- finds the null values and gives statistical analysis of the number of values .

In [ ]:
# ndf=df.isnull()
# ndf.describe()

### cleansing the data

- count of null values

In [ ]:
# df.isna().sum()

In [ ]:
df_pyspark1 = df_pyspark.na.drop("any")

In [ ]:
df_pyspark1.count()

In [ ]:
# df = df.replace('0', np.nan)

- drop the null values

In [ ]:
# df=df.dropna()

In [ ]:
# df.isna().sum()

- statistical description after removing the null values

In [ ]:
# df.info()

# Data Preprocessing

#### renaming the columns names

In [ ]:
# df.columns

In [ ]:
df_pyspark1.columns

In [ ]:
# df = df.rename(columns={"PU_latitude":"pickup_latitude", "PU_longitude":"pickup_longitude", "DO_latitude":"dropoff_latitude", "DO_longitude":"dropoff_longitude", "tpep_pickup_datetime":"pickup_datetime", "tpep_dropoff_datetime":"dropoff_datetime"})
# df.head()

In [ ]:
df_pyspark1.withColumnRenamed()

#### remove the locations outside the New York City

- minimum and maximum latitude and longitude of NYC

In [ ]:
def min_max():
    print(df['pickup_longitude'].max(),df['pickup_longitude'].min())
    print(df['pickup_latitude'].max(),df['pickup_latitude'].min())

    print(df['dropoff_longitude'].max(),df['dropoff_longitude'].min())
    print(df['dropoff_latitude'].max(),df['dropoff_latitude'].min())
min_max()

- remove where pickup and drop location are same

In [ ]:
print(df[(df['pickup_longitude']==df['dropoff_longitude']) & (df['pickup_latitude']==df['dropoff_latitude'])].shape)
df=df[~((df['pickup_longitude']==df['dropoff_longitude']) & (df['pickup_latitude']==df['dropoff_latitude']))]

- shape of data frame 

In [ ]:
df.shape

- boundary for new york latitude longitude...

In [ ]:
boundary={"min_lat":40.573143,
         "min_lang":-74.252193,
         "max_lat":41.709555,
         "max_lang":-72.986532}

- count of locations outside the new york city

In [ ]:
print(df[(df['pickup_latitude']<=boundary['min_lat']) | (df['pickup_latitude']>=boundary['max_lat'])].shape[0])
print(df[(df['pickup_longitude']<=boundary['min_lang']) | (df['pickup_longitude']>=boundary['max_lang'])].shape[0])
print(df[(df['dropoff_latitude']<=boundary['min_lat']) | (df['dropoff_latitude']>=boundary['max_lat'])].shape[0])
print(df[(df['dropoff_longitude']<=boundary['min_lang']) | (df['dropoff_longitude']>=boundary['max_lang'])].shape[0])

- removing the locations which are outside the boundaries

In [ ]:
df=df[~((df['pickup_latitude']<=boundary['min_lat']) | (df['pickup_latitude']>=boundary['max_lat']))]
df=df[~((df['pickup_longitude']<=boundary['min_lang']) | (df['pickup_longitude']>=boundary['max_lang']))]

df=df[~((df['dropoff_latitude']<=boundary['min_lat']) | (df['dropoff_latitude']>=boundary['max_lat']))]
df=df[~((df['dropoff_longitude']<=boundary['min_lang']) | (df['dropoff_longitude']>=boundary['max_lang']))]

df.shape

### plotting the pickup latitude and pickup longitude

In [ ]:
df.plot(y='pickup_latitude',x='pickup_longitude',kind="scatter",alpha=0.7,s=0.02)

city_long_border = (-74.03, -73.75)
city_lat_border = (40.63, 40.85)
plt.title("Pickups Data")

plt.ylim(city_lat_border)
plt.xlim(city_long_border)
plt.show()

### plot of dropoff latitude and dropoff longitude

In [ ]:
df.plot(y='dropoff_latitude',x='dropoff_longitude',kind="scatter",alpha=0.5,s=0.02)
city_long_border = (-74.03, -73.75)
city_lat_border = (40.63, 40.85)
plt.title("Dropoff Data")

plt.ylim(city_lat_border)
plt.xlim(city_long_border)
plt.show()

### Passengers Count Data

- max and min count of passengers

In [ ]:
df['passenger_count']=df['passenger_count'].astype(int)
print(df['passenger_count'].max())
print(df['passenger_count'].min())

In [ ]:
sns.countplot(y='passenger_count',data=df)

- removing records with passengers more than 6

In [ ]:
df=df[~((df['passenger_count']>6))]
df.shape

- removing passengers with negative value as a count

In [ ]:
df[df['fare_amount']<=0]['fare_amount'].count()
df=df[~(df['fare_amount']<=0)]
df.shape

# Feature Engineering

### checking the data types of the columns

In [ ]:
df.info()

- changing the data type from string to timestamp

In [ ]:
type(df['pickup_datetime'].iloc[0])

In [ ]:
df['pickup_datetime']=pd.to_datetime(df['pickup_datetime'])
type(df['pickup_datetime'].iloc[0])

In [ ]:
df['dropoff_datetime']=pd.to_datetime(df['dropoff_datetime'])
type(df['dropoff_datetime'].iloc[0])

### adding the new column for day of week, hour of day, month and year, trip duration

In [ ]:
df['pickup_day']=df['pickup_datetime'].dt.dayofweek
df['dropoff_day']=df['dropoff_datetime'].dt.dayofweek

In [ ]:
df['pickup_day_no']=df['pickup_datetime'].dt.weekday
df['dropoff_day_no']=df['dropoff_datetime'].dt.weekday

In [ ]:
df['pickup_hour']=df['pickup_datetime'].dt.hour
df['dropoff_hour']=df['dropoff_datetime'].dt.hour

In [ ]:
df['pickup_month']=df['pickup_datetime'].dt.month
df['dropoff_month']=df['dropoff_datetime'].dt.month

In [ ]:
df['pickup_year']=df['pickup_datetime'].dt.year
df['dropoff_year']=df['dropoff_datetime'].dt.year

In [ ]:
df['trip_duration'] = df.dropoff_datetime - df.pickup_datetime 
df.head()

- pickup_day and dropoff_day which will contain the name of the day on which the ride was taken.
- pickup_day_no and dropoff_day_no which will contain the day number instead of characters with Monday=0 and Sunday=6.
- pickup_hour and dropoff_hour with an hour of the day in the 24-hour format.
- pickup_month and dropoff_month with month number with January=1 and December=12.

- changing the data type of trip duration

In [ ]:
type(df.trip_duration)

In [ ]:
df['trip_duration']=pd.to_timedelta(df['trip_duration'])

In [ ]:
type(df.trip_duration.iloc[0])

In [ ]:
for i in range(len(df)):
    
    df.trip_duration.iloc[i] = df.trip_duration.iloc[i].seconds

In [ ]:
df.head()

### function to determine at what time of day ride was taken

In [ ]:
def time_of_day(x):
    if x in range(6,12):
        return 'Morning'
    elif x in range(12,16):
        return 'Afternoon'
    elif x in range(16,22):
        return 'Evening'
    else:
        return 'Late night'

In [ ]:
df['pickup_timeofday']=df['pickup_hour'].apply(time_of_day)
df['dropoff_timeofday']=df['dropoff_hour'].apply(time_of_day)

In [ ]:
df.head()

### mid night trip column

- 1 for YES
- 0 for NO
- Let's add the feature called mid night trip usually fare during night hours are high compare to day time
- The trip between night 11'o clock to morning 5'o clock are labelled mid night trip

In [ ]:
df['mid_night_trip']=((df['pickup_timeofday']=='Late night')).astype(int)
df.head()

### 'Rush hour trip' column

- New York City has rush hour means heavy traffic from 4 PM to 8 PM
- So fare rates may differ from other %time

In [ ]:
df['rush_hour_trip']=((df['pickup_timeofday']=='Evening')).astype(int)
df.head()

### Trips in snow season

- During snow season the taxi is very less available compare to other months i.e. taxi fare during snow season is high.
- NYC has the snow season from December to March but December and March has less snow compare to Jan and Feb
- So we will check if trip was in month of Jan and Feb it is during snow season

In [ ]:
df['snow_season']=((df['pickup_month']==1) | (df['pickup_month']==2)).astype(int)
df.head()

### Distance calculator between pickup and drop coordinates

- Distance and the fare amount has the direct correlation if distance increase fare will be increase
- We will use python's geopy library to add column trip_distance which will have distance between two coordinates

###### using geopy library

In [ ]:
from geopy.distance import geodesic

def distance_calculate(lat,long,drop_lat,drop_long):    
    newport_ri = (lat,long)
    cleveland_oh = (drop_lat,drop_long)
    dist=geodesic(newport_ri, cleveland_oh).miles
    return dist

In [ ]:
df['distance_lib']=list(map(distance_calculate,df['pickup_latitude'],df['pickup_longitude'],df['dropoff_latitude'],df['dropoff_longitude']))
df.head()

###### using haverstine dsitance formula

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine_distance(row):
#     lon1, lat1, lon2, lat2):
    """
    Calculate the circle distance between two points in lat and lon
    on the earth (specified in decimal degrees)
    returning distance in miles
    """
    # need to convert decimal degrees to radians 
    # a unit of angle, equal to an angle at the center of a circle whose arc is equal in length to the radius.
    lon1, lat1, lon2, lat2 = row['pickup_longitude'], row['pickup_latitude'], row['dropoff_longitude'], row['dropoff_latitude']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3965 # Radius of earth in kilometers (6371 for kms). Use 3956 for miles
    return c * r
#applying to the dataset
df['haversine_distance'] = df.apply(haversine_distance, axis=1)

In [ ]:
df.head()

###### comparing both the calculated distances and getting it's mean as a value

In [ ]:
df1 = df[['distance_lib', 'haversine_distance']]
df1.head()

In [ ]:
df['trip_distance'] = (df['distance_lib']+df['haversine_distance'])/2
df['trip_distance'].head()

In [ ]:
df.columns

### Mapping

###### Convert specific pickup and drop Coordinates into location

- We have set the boundary for the JFK Airport, Manhattan, Brooklyn, LaGuardia, Bronx

In [ ]:
#JFK Airport 
JFK={"min_long":-73.81,
    "min_lat":40.63,
    "max_long":-73.77,
    "max_lat":40.67}

#Manhattan 
Manhattan={
    "min_long":-74.02,
    "min_lat":40.70,
    "max_long":-73.93,
    "max_lat":40.85}

#Brooklyn
Brooklyn={
    "min_long":-74.01,
    "min_lat":40.62,
    "max_long":-73.96,
    "max_lat":40.70 }

#LaGuardia
LaGuardia={
    "min_long":-73.89,
    "min_lat":40.76,
    "max_long":-73.85,
    "max_lat":40.79}

#Bronx
Bronx={
    "min_long":-73.935,
    "min_lat":40.79,
    "max_long":-73.88,
    "max_lat":40.89}

###### number of pickup from the lcoation

In [ ]:
def pickup_place(place,train):
    data=train[((train['pickup_latitude']>= place['min_lat']) & (train['pickup_latitude']<= place['max_lat']) & (train['pickup_longitude']>= place['min_long']) & (train['pickup_longitude']<= place['max_long']))]    
    return data

print("JFK Pickup Total " , pickup_place(JFK,df).shape)
print("Manhattan Pickup Total " , pickup_place(Manhattan,df).shape)
print("Brroklyn Pickup Total " , pickup_place(Brooklyn,df).shape)
print("Bronx Pickup Total " , pickup_place(Bronx,df).shape)
print("LaGuardia Pickup Total " , pickup_place(LaGuardia,df).shape)

###### number of dropoff from the location

In [ ]:
def drop_place(place,train):
    data=train[((train['dropoff_latitude']>= place['min_lat']) & (train['dropoff_latitude']<= place['max_lat']) & (train['dropoff_longitude']>= place['min_long']) & (train['dropoff_longitude']<= place['max_long']))]    
    return data

print("JFK Dropoff Total " , drop_place(JFK,df).shape)
print("Manhattan Dropoff Total " , drop_place(Manhattan,df).shape)
print("Brroklyn Dropoff Total " , drop_place(Brooklyn,df).shape)
print("Bronx Dropoff Total " , drop_place(Bronx,df).shape)
print("LaGuardia Dropoff Total " , drop_place(LaGuardia,df).shape)

###### function for pickup

In [ ]:
def plotPickupAreaEachPlace(Area,data,ax):
    d=pickup_place(Area,data)
    d.plot(y='pickup_latitude',x='pickup_longitude',kind="scatter",alpha=0.7,s=0.02,ax=ax,figsize=(14,6))
    city_long_border = (-74.03, -73.75)
    city_lat_border = (40.63, 40.85)
    plt.title("Pickups")

    plt.ylim(city_lat_border)
    plt.xlim(city_long_border)
    plt.show()

In [ ]:
ax=plt.subplot(2,3,1)
plotPickupAreaEachPlace(JFK,df,ax)
ax=plt.subplot(2,3,2)
plotPickupAreaEachPlace(Manhattan,df,ax)
ax=plt.subplot(2,3,3)
plotPickupAreaEachPlace(Brooklyn,df,ax)
ax=plt.subplot(2,3,4)
plotPickupAreaEachPlace(Bronx,df,ax)
ax=plt.subplot(2,3,5)
plotPickupAreaEachPlace(LaGuardia,df,ax)

###### function for drop off from the boroughs

In [ ]:
def plotDropAreaEachPlace(Area,data,ax):
    d=drop_place(Area,data)
    d.plot(y='dropoff_latitude',x='dropoff_longitude',kind="scatter",alpha=0.7,s=0.02,ax=ax,figsize=(14,6))
    city_long_border = (-74.03, -73.75)
    city_lat_border = (40.63, 40.85)
    plt.title("Pickups")

    plt.ylim(city_lat_border)
    plt.xlim(city_long_border)
    plt.show()

In [ ]:
ax=plt.subplot(2,3,1)
plotDropAreaEachPlace(JFK,df,ax)
ax=plt.subplot(2,3,2)
plotDropAreaEachPlace(Manhattan,df,ax)
ax=plt.subplot(2,3,3)
plotDropAreaEachPlace(Brooklyn,df,ax)
ax=plt.subplot(2,3,4)
plotDropAreaEachPlace(Bronx,df,ax)
ax=plt.subplot(2,3,5)
plotDropAreaEachPlace(LaGuardia,df,ax)

place_mapping function which will take lat,lang and bounday area varibale it will map to particular area

It will return 1 if particular lat,lang's are within specific boundary.

e.g pickup coordinates which are within boundary for JFK Airport said to be JFK pickup trip and vice versa for drop

In [ ]:
def place_mapping(lat,lang,place_area):
    if (lat>= place_area['min_lat']) & (lat<= place_area['max_lat']) & (lang>= place_area['min_long']) & (lang<= place_area['max_long']):
        return 1
    return 0

In [ ]:
df['pickup_manhattan']=df.apply(lambda row:place_mapping(row['pickup_latitude'],row['pickup_longitude'],Manhattan),axis=1)
df['pickup_jfk']=df.apply(lambda row:place_mapping(row['pickup_latitude'],row['pickup_longitude'],JFK),axis=1)
df['pickup_brooklyn']=df.apply(lambda row:place_mapping(row['pickup_latitude'],row['pickup_longitude'],Brooklyn),axis=1)
df['pickup_bronx']=df.apply(lambda row:place_mapping(row['pickup_latitude'],row['pickup_longitude'],Bronx),axis=1)
df['pickup_laGuardia']=df.apply(lambda row:place_mapping(row['pickup_latitude'],row['pickup_longitude'],LaGuardia),axis=1)

In [ ]:
df['drop_manhattan']=df.apply(lambda row:place_mapping(row['dropoff_latitude'],row['dropoff_longitude'],Manhattan),axis=1)
df['drop_jfk']=df.apply(lambda row:place_mapping(row['dropoff_latitude'],row['dropoff_longitude'],JFK),axis=1)
df['drop_brooklyn']=df.apply(lambda row:place_mapping(row['dropoff_latitude'],row['dropoff_longitude'],Brooklyn),axis=1)
df['drop_bronx']=df.apply(lambda row:place_mapping(row['dropoff_latitude'],row['dropoff_longitude'],Bronx),axis=1)
df['drop_laGuardia']=df.apply(lambda row:place_mapping(row['dropoff_latitude'],row['dropoff_longitude'],LaGuardia),axis=1)

Airport trip has the fixed rate so it is good to have a column which show whether trip is airport trip or not

In [ ]:
def isAirportTrip(pickupJFK,dropJFK,pickupLaG,dropLaG):    
    if pickupJFK==1 or dropJFK==1 or pickupLaG==1 or dropLaG==1:
        return 1
    return 0

In [ ]:
df['isAirportTrip']=df.apply(lambda row:isAirportTrip(row['pickup_jfk'],row['drop_jfk'],row['pickup_laGuardia'],row['drop_laGuardia']),axis=1)

Till here we have pre-processed our data and feature engineering done

# Save the pre-processed and feature_eng data in CSV file

In [ ]:
from secrets import access_key, secret_access_key

In [ ]:
client = boto3.client('s3',
                        aws_access_key_id = access_key,
                        aws_secret_access_key = secret_access_key)

In [ ]:
df.to_csv("./feature_model.csv",index=False)

In [ ]:
client.upload_file('./feature_model.csv', 'nyc-taxi-system', 'feature_model.csv')
print("Uploaded on AWS")
os.remove('./feature_model.csv')
print("File removed")

### Read Pre-processed file

In [ ]:
path = 's3://nyc-taxi-system/feature_model.csv'

In [ ]:
df=pd.read_csv(path)
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

### Univariate Analysis

### Target Variable

In [ ]:
sns.histplot(df['trip_duration'],kde=False,bins=20)

- The histogram is really skewed as we can see.

In [ ]:
sns.boxplot(df['trip_duration'])

- not many outliers

In [ ]:
df['trip_duration'].sort_values(ascending=False)

#### Vendor Id

In [ ]:
sns.countplot(x='VendorID',data=df)

- we can see there is significant difference between these two vendors trip vcount

#### Passenger Count

In [ ]:
df.passenger_count.value_counts()

In [ ]:
sns.countplot(x='passenger_count',data=df)

- the highest amount of trips are with 1 passenger.

#### Store and Forward Flag

In [ ]:
df['store_and_fwd_flag'].value_counts(normalize=True)

- We see there are only 1% of trips that were stored before forwarding.

#### Distance

In [ ]:
df['trip_distance'].value_counts()

- The reasons for 0 km distance can be:
     - The dropoff location couldn’t be tracked.
     - The driver deliberately took this ride to complete a target ride number.
     - The passengers canceled the trip.

### Trips per day

In [ ]:
figure,(ax1,ax2)=plt.subplots(ncols=2,figsize=(20,5))

ax1.set_title('Pickup Days')
ax=sns.countplot(x="pickup_day",data=df,ax=ax1)

ax2.set_title('Dropoff Days')
ax=sns.countplot(x="dropoff_day",data=df,ax=ax2)

- We see Fridays are the busiest days followed by Saturdays. That is probably because it’s weekend.

### trips per hour

In [ ]:
figure,(ax9,ax10)=plt.subplots(ncols=2,figsize=(20,5))

ax9.set_title('Pickup Days')
ax=sns.countplot(x="pickup_hour",data=df,ax=ax9)

ax10.set_title('Dropoff Days')
ax=sns.countplot(x="dropoff_hour",data=df,ax=ax10)

- We see the busiest hours are 6:00 pm to 7:00 pm and that makes sense as this is the time when people return from their offices.

### Trips per Time of Day

In [ ]:
figure,(ax3,ax4)=plt.subplots(ncols=2,figsize=(20,5))

ax3.set_title('Pickup Time of Day')
ax=sns.countplot(x="pickup_timeofday",data=df,ax=ax3)

ax4.set_title('Dropoff Time of Day')
ax=sns.countplot(x="dropoff_timeofday",data=df,ax=ax4)

- As we saw above, evenings are the busiest.

### trips per month

In [ ]:
figure,(ax11,ax12)=plt.subplots(ncols=2,figsize=(20,5))
ax11.set_title('Pickup Month')
ax=sns.countplot(x="pickup_month",data=df,ax=ax11)
ax12.set_title('Dropoff Month')
ax=sns.countplot(x="dropoff_month",data=df,ax=ax12)

- There is not much difference in the number of trips across months.

### Bivariate Analysis

- Bivariate Analysis involves finding relationships, patterns, and correlations between two variables.## 

### Trip Duration per Vendor

In [ ]:
sns.barplot(y='trip_duration',x='VendorID',data=df,estimator=np.mean)

- Vendor id 2 takes longer trips as compared to vendor 1.

### Trip Duration per Store and Forward Flag

In [ ]:
sns.catplot(y='trip_duration',x='store_and_fwd_flag',data=df,kind="strip")

- Trip duration is generally longer for trips whose flag was not stored

### Trip Duration per passenger count

In [ ]:
sns.catplot(y='trip_duration',x='passenger_count',data=df,kind="strip")

- There is no visible relation between trip duration and passenger count.

### Trip Duration per hour

In [ ]:
sns.lineplot(x='pickup_hour',y='trip_duration',data=df)

- We see the trip duration is the maximum around 3 pm which may be because of traffic on the roads.

- Trip duration is the lowest around 6 am as streets may not be busy.

### Trip duration per time of day

In [ ]:
sns.lineplot(x='pickup_timeofday',y='trip_duration',data=df)

- As we saw above, trip duration is the maximum in the afternoon and lowest between late night and morning.

### Trip duration per Day of Week

In [ ]:
sns.lineplot(x='pickup_day_no',y='trip_duration',data=df)

- Trip duration is the longest on Thursdays closely followed by Fridays

### Trip duration per month

In [ ]:
sns.lineplot(x='pickup_month',y='trip_duration',data=df)

- From February, we can see trip duration rising every month.

### Distance and Vendor

In [ ]:
sns.barplot(y='trip_distance',x='VendorID',data=df,estimator=np.mean)

- The distribution for both vendors is very similar.

### Distance and Store and Forward Flag

In [ ]:
sns.catplot(y='trip_distance',x='store_and_fwd_flag',data=df,kind="strip")

- We see for longer distances the trip is not stored.

### Distance per passenger count

In [ ]:
sns.catplot(y='trip_distance',x='passenger_count',data=df,kind="strip")

- We see some of the longer distances are covered by either 1 or 2 or 4 passenger rides

### Distance per day of week

In [ ]:
sns.lineplot(x='pickup_day_no',y='trip_distance',data=df)

- Distances are longer on Sundays probably because it’s weekend.
- Monday trip distances are also quite high.
- This probably means that there can be outstation trips on these days and/or the streets are busier.

### Distance per hour of day

In [ ]:
sns.lineplot(x='pickup_hour',y='trip_distance',data=df)

- Distances are the longest around 5 am.

### Distance per time of day

In [ ]:
sns.lineplot(x='pickup_timeofday',y='trip_distance',data=df)

- As seen above also, distances being the longest during late night or it maybe called as early morning too.
- This can probably point to outstation trips where people start early for the day.

### Distance per month

In [ ]:
sns.lineplot(x='pickup_month',y='trip_distance',data=df)

- As we also saw during trip duration per month, similarly trip distance is the lowest in February and the maximum in June

### Passenger Count and Vendor id

In [ ]:
sns.barplot(y='passenger_count',x='VendorID',data=df)

- This shows that vendor 2 generally carries 2 passengers while vendor 1 carries 1 passenger rides.

### Trip Duration and Distance

In [ ]:
sns.relplot(y=df.trip_distance,x='trip_duration',data=df)

- We can see there are trips which trip duration as short as 0 seconds and yet covering a large distance. And, trips with 0 km distance and long trip durations.

In [ ]:
df[df.trip_distance==0].head()

In [ ]:
df[df.trip_distance==0].tail()

- We can see even though distance is recorded as 0 but trip duration is definitely more.
    - One reason can be that the dropoff coordinates weren’t recorded.
    - Another reason one can think is that for short trip durations, maybe the passenger changed their mind and cancelled the ride after some time.

### Data Visualization

The next step is to get insight to data.

Remove any noisy or outlier data and understand data distribution

In [ ]:
df.columns

#### distance travelled by hour by days

In [ ]:
sns.set(font_scale=1.3)
g = sns.factorplot('pickup_hour', 
                   'haversine_distance', 
                   hue = 'pickup_day', 
                   estimator = np.mean, 
                   data = df, 
                   size = 8, 
                   aspect = 2, 
                    ci=None,
                   legend_out=False)
plt.title('Average Duration by Hour of Day and Day of Week \n',weight='bold', size = 20)
plt.xlabel('start hour', size = 18,weight = 'bold')
plt.ylabel('distance', size = 18,weight = 'bold')
g.set_xticklabels(rotation=60)

####  count of trip per year, per month, per weekdays, per hour

In [ ]:
plt.figure(figsize=(12,8))
plt.subplot(2,2,1)
sns.countplot(y='pickup_year', data=df)
plt.subplot(2,2,2)
sns.countplot(x='pickup_month', data=df,palette='Set3')
plt.subplot(2,2,3)
sns.countplot(x='pickup_hour', data=df,palette='Set2')
plt.subplot(2,2,4)
sns.countplot(y='pickup_day', data=df)

plt.show()

####  count as per mid night trip, per rush hour trip, per passenger count, per snow season

In [ ]:
plt.figure(figsize=(12,8))

plt.subplot(2,2,1)
sns.countplot(y='mid_night_trip', data=df,palette='Set1')

plt.subplot(2,2,2)
sns.countplot(x='rush_hour_trip', data=df,palette='Set2')

plt.subplot(2,2,3)
sns.countplot(x='passenger_count', data=df,palette='Set3')

plt.subplot(2,2,4)
sns.countplot(x='snow_season', data=df)

plt.show()

####  count as per pick weekday and as per airport trip

In [ ]:
plt.figure(figsize=(12,8))

plt.subplot(2,2,1)
sns.countplot(x='pickup_day',data=df,palette='Set3')

plt.subplot(2,2,2)
sns.countplot(x='isAirportTrip',data=df,palette='Set2')

####  Pickup Location Count Plot as per region

In [ ]:
plt.figure(figsize=(16,7))
plt.subplot(2,3,1)
sns.countplot(x='pickup_jfk',data=df)
plt.subplot(2,3,2)
sns.countplot(x='pickup_manhattan',data=df)
plt.subplot(2,3,3)
sns.countplot(x='pickup_bronx',data=df)
plt.subplot(2,3,4)
sns.countplot(x='pickup_brooklyn',data=df)
plt.subplot(2,3,5)
sns.countplot(x='pickup_laGuardia',data=df)
# plt.show()

####  Drop Location Count Plot as per region

In [ ]:
plt.figure(figsize=(16,7))
plt.subplot(2,3,1)
sns.countplot(x='drop_jfk',data=df)
plt.subplot(2,3,2)
sns.countplot(x='drop_manhattan',data=df)
plt.subplot(2,3,3)
sns.countplot(x='drop_bronx',data=df)
plt.subplot(2,3,4)
sns.countplot(x='drop_brooklyn',data=df)
plt.subplot(2,3,5)
sns.countplot(x='drop_laGuardia',data=df)
plt.show()

####  Fare amount vs Trip distance

In [ ]:
plt.figure(figsize=(8,8))

# fare amount density
plt.subplot(2,2,1)
sns.distplot(df['fare_amount'],kde=True)

# trip distance density 
plt.subplot(2,2,2)
sns.distplot(df['trip_distance'],kde=True)

# fare amount vs trip distance
df.plot(x='fare_amount',y='trip_distance',kind='scatter')

####  cancel ride desnity

In [ ]:
cancel_ride=df[(df['fare_amount']>=0) & (df['passenger_count']==0)]
sns.distplot(cancel_ride['fare_amount'],kde=True)

####  log of fare amount vs density 

In [ ]:
sns.distplot(np.log(df['fare_amount']),kde=True)

####  Airport vs Non Aiport Trip Fare Distributinon

In [ ]:
df_airport=df[df['isAirportTrip']==1]
df_non_airport=df[df['isAirportTrip']==0]

fig = plt.figure(figsize=(6,4))
sns.distplot(df_airport['fare_amount'],kde=True,color="y")
sns.distplot(df_non_airport['fare_amount'],kde=True)
fig.legend(labels=['Airport Fare','Non Airport Fare'])
plt.show()

####  Different Area Fare Distribution

In [ ]:
df_manhattan=df[(df['pickup_manhattan']==1) | (df['drop_manhattan']==1)]
df_jfk=df[(df['pickup_jfk']==1) | (df['drop_jfk']==1)]
df_bronx=df[(df['pickup_bronx']==1) | (df['drop_bronx']==1)]
df_brooklyn=df[(df['pickup_brooklyn']==1) | (df['drop_brooklyn']==1)]
df_LGD=df[(df['pickup_laGuardia']==1) | (df['drop_laGuardia']==1)]

In [ ]:
fig = plt.figure(figsize=(6,4))

sns.distplot(df_manhattan['fare_amount'],kde=True)
sns.distplot(df_jfk['fare_amount'],kde=True)
sns.distplot(df_bronx['fare_amount'],kde=True)
sns.distplot(df_brooklyn['fare_amount'],kde=True)
sns.distplot(df_LGD['fare_amount'],kde=True)
fig.legend(labels=['Manhattan Fare','JFK Fare','Bronx Fare','Brooklyn Fare','LaGuardia Fare'])
plt.show()

####  Fare Distribution of Manhattan

In [ ]:
df_man=df[ (df['pickup_manhattan'] ==1) & (df['drop_manhattan'] ==1)]
sns.distplot(df_man['fare_amount'],kde=True)

In [ ]:
df_man['fare_amount'].sum()/df_man.shape[0]
df=df[~((df['pickup_manhattan'] ==1) & (df['drop_manhattan'] ==1) & (df['fare_amount'] > 20))]

####  Fare Distribution at Different Time Zone

In [ ]:
df_snow=df[df['snow_season']==1]
df_non_snow=df[df['snow_season']==0]

fig = plt.figure(figsize=(7,6))
ax=plt.subplot(2,2,1)
sns.distplot(np.log(df_snow['fare_amount']),ax=ax)
sns.distplot(np.log(df_non_snow['fare_amount']),ax=ax)


df_mid=df[df['mid_night_trip']==1]
df_non_mid=df[df['mid_night_trip']==0]
ax=plt.subplot(2,2,2)
sns.distplot(np.log(df_mid['fare_amount']),ax=ax)
sns.distplot(np.log(df_non_mid['fare_amount']),ax=ax)

df_rush_hour=df[df['rush_hour_trip']==1]
df_non_rush_hour=df[df['rush_hour_trip']==0]
ax=plt.subplot(2,2,3)
sns.distplot(np.log(df_rush_hour['fare_amount']),ax=ax)
sns.distplot(np.log(df_non_rush_hour['fare_amount']),ax=ax)

### Correlation

- corr() is used for correlation

In [ ]:
df.corr()

- heatmap to look for variable dependencies

In [ ]:
sns.heatmap(df.corr())

- shows all columns

In [ ]:
df.columns

- show number of records and number of columns 

In [ ]:
df.shape

In [ ]:
df.info()

### PCA

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Profit is the dependent column
y = df['total_amount']

# start with nothing
x_names = []

# the previous r2
previous_r2 = -1

# list of all columns
columns = ['VendorID', 'passenger_count',
       'trip_distance', 'RatecodeID', 'PULocationID',
       'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'pickup_latitude',
       'pickup_longitude', 'dropoff_latitude',
       'dropoff_longitude', 'pickup_day', 'dropoff_day', 'pickup_day_no',
       'dropoff_day_no', 'pickup_hour', 'dropoff_hour', 'pickup_month',
       'dropoff_month', 'pickup_year', 'dropoff_year', 'trip_duration',
       'mid_night_trip',
       'rush_hour_trip', 'snow_season', 'distance_lib', 'haversine_distance',
       'pickup_manhattan', 'pickup_jfk', 'pickup_brooklyn', 'pickup_bronx',
       'pickup_laGuardia', 'drop_manhattan', 'drop_jfk', 'drop_brooklyn',
       'drop_bronx', 'drop_laGuardia', 'isAirportTrip']

# choose one col at a time and check its significance
for col in columns:
    
    # this will hold all the variables need to pass to the model
    temp_variables = []
    
    # add previously selected variable(s)
    temp_variables.extend(x_names)
    
    # add the current column
    temp_variables.append(col)
    
    print(f"selected columns {temp_variables}")
    
    # get all the variables in the form of a df
    x = df[temp_variables]
    
    # create model 
    model = LinearRegression()
    
    # train the model
    model.fit(x, y)
    
    # predict the values
    predictions = model.predict(x)
    
    # evaluate the model
    r2 = r2_score(y, predictions)

    print(f"previous r2 = {previous_r2}, current r2 = {r2}")
    
    # check if this is the first variable
    if previous_r2 == -1:
        previous_r2 = r2
        x_names.append(col)
        print(rf"since this is the first column, add ({col}) it to the list")
    else:
        # check if this r2 is greater than the previous one
        # if r2 > previous_r2:
        
        # check if the r2 difference is significant
        print(f"r2 - previous_r2 = {r2 - previous_r2}")
        if (r2 - previous_r2) >= 0.05:
            previous_r2 = r2
            x_names.append(col)
            print(f"found col: {col} increasing the r2 score")
        
        
print(f"finally selected columns = {x_names}")

### Separate the independent and dependent relationship

- independent variables

In [ ]:
x = df.drop(['pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'RatecodeID', 'store_and_fwd_flag', 'PULocationID',
       'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'congestion_surcharge', 'PU_zone', 'PU_borough', 'pickup_latitude',
       'pickup_longitude', 'DO_zone', 'DO_borough', 'dropoff_latitude',
       'dropoff_longitude', 'pickup_day', 'dropoff_day', 'pickup_day_no',
       'dropoff_day_no', 'pickup_hour', 'dropoff_hour', 'pickup_month',
       'dropoff_month', 'pickup_year', 'dropoff_year', 'trip_duration',
       'pickup_timeofday', 'dropoff_timeofday', 'mid_night_trip',
       'rush_hour_trip', 'snow_season', 'distance_lib', 'haversine_distance',
       'pickup_manhattan', 'pickup_jfk', 'pickup_brooklyn', 'pickup_bronx',
       'pickup_laGuardia', 'drop_manhattan', 'drop_jfk', 'drop_brooklyn',
       'drop_bronx', 'drop_laGuardia', 'isAirportTrip', 'total_amount'], axis=1).values

In [ ]:
x

- dependent variable

In [ ]:
y = df['total_amount']

In [ ]:
y

## LINEAR REGRESSION

#### Split the data into train and tests sets

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,test_size=0.2,random_state=864)

### Import linear regression algorithm from scikit

In [ ]:
from sklearn.linear_model import LinearRegression
# creating model using LinearRegression algorithm
model = LinearRegression()

### Creating Model

In [ ]:
# creating model 
model.fit(x_train,y_train)

### Testing the model

In [ ]:
# importing libraries from sklearn
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

# storing predicted values in predictions
predictions=model.predict(x_test)

# r2_score
print(f"r2_score : {r2_score(y_test,predictions)}")
print(f"mean absolute error : {mean_absolute_error(y_test,predictions)}")
print(f"mean squared error : {mean_squared_error(y_test,predictions)}")

In [ ]:
import pickle

# create the file to store the model
# w: write
# b: binary data
file = open('taxi_fare_linear.pkl', 'wb')

# store the model using pickle
pickle.dump(model, file)

# save the file
file.close()

## XG Boost

#### SPlitting the dataset into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,test_size=0.2,random_state=563)

#### Importing XGBoost algorithm from scikit

In [ ]:
from xgboost import XGBRegressor
# creating model using LinearRegression algorithm
model_2 = XGBRegressor()

In [ ]:
model_2.get_params()

In [ ]:
# parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
#               'objective':['reg:linear'],
#               'learning_rate': [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ], #so called `eta` value
#               'max_depth': [ 3, 4, 5, 6, 8, 10, 12, 15],
#               'gamma' : [0.0, 0.1, 0.2 , 0.3, 0.4],
#               'eta':[.03], 
#               'eval_metric':['rmse'],
#               'min_child_weight': [1, 3, 5, 7],
#               'silent': [1],
#               'subsample': [0.7],
#               'colsample_bytree': [0.3, 0.4, 0.5 , 0.7],
#               'n_estimators': [786]}

# paramters = {
#      "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
#      "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12],
#      "min_child_weight" : [ 1, 3, 5, 7 ],
#      "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
#      "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
# }

In [ ]:
# from sklearn.model_selection import GridSearchCV

In [ ]:
# xgb_grid = GridSearchCV(model_2,
#                         parameters,
#                         cv = 2,
#                         n_jobs = 5,
#                         verbose=True)

#### Creating the model

In [ ]:
# creating model 
model_2.fit(x_train,y_train)

In [ ]:
# print(xgb_grid.best_score_)
# print(xgb_grid.best_params_)

#### Testing the model

In [ ]:
# importing libraries from sklearn
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

# storing predicted values in predictions
predictions=model_2.predict(x_test)

# r2_score
print(f"r2_score : {r2_score(y_test,predictions)}")
print(f"mean absolute error : {mean_absolute_error(y_test,predictions)}")
print(f"mean squared error : {mean_squared_error(y_test,predictions)}")

In [ ]:
import pickle

# create the file to store the model
# w: write
# b: binary data
file = open('taxi_fare_XG.pkl', 'wb')

# store the model using pickle
pickle.dump(model_2, file)

# save the file
file.close()

## RANDOM FOREST

#### Splitting the dataset into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,test_size=0.2,random_state=563)
# 563

#### Importing random forest algorithm from scikit

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# creating model using Random Forest algorithm
model_3 = RandomForestRegressor()

In [ ]:
model_3.get_params()

In [ ]:
# parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
#               'objective':['reg:linear'],
#               'learning_rate': [.03, 0.05, .07], #so called `eta` value
#               'max_depth': [5, 6, 7],
#               'min_child_weight': [4],
#               'silent': [1],
#               'subsample': [0.7],
#               'colsample_bytree': [0.7],
#               'n_estimators': [500]}

In [ ]:
# param_grid = { 
#     'n_estimators': [300, 500, 700],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : [4,5,6,7,8],
# }

In [ ]:
# from sklearn.model_selection import GridSearchCV

In [ ]:
# rf_grid = GridSearchCV(model_3,
#                         param_grid,
#                         cv = 2,
#                         n_jobs = 5,
#                         verbose=True)

#### creating the model

In [ ]:
# creating model 
model_3.fit(x_train,y_train)

### Testing the model

In [ ]:
# importing libraries from sklearn
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

# storing predicted values in predictions
predictions=model_3.predict(x_test)

# r2_score
print(f"r2_score            : {r2_score(y_test,predictions)}")
print(f"mean absolute error : {mean_absolute_error(y_test,predictions)}")
print(f"mean squared error  : {mean_squared_error(y_test,predictions)}")

In [ ]:
value=np.array([2, 2.10])

In [ ]:
value

In [ ]:
model_3.predict([value])

In [ ]:
import pickle

# create the file to store the model
# w: write
# b: binary data
file = open('taxi_fare_RANDOM.pkl', 'wb')

# store the model using pickle
pickle.dump(model_3, file)

# save the file
file.close()